In [1]:
import pandas as pd
import numpy as np
import datetime as datetime

Done importing

Reading in raw dataframe. Adding in a date column where each entry has a date attatched to it

In [7]:
prices_df = pd.read_excel('USO_Historical_Data_30_Minute.xlsx', sheet_name=None, index_col=False)

In [69]:
def clean_date(sheet_dic):
    # Creating dataframe out of input
    combined_df = pd.DataFrame()
    for key in sheet_dic:
        combined_df = pd.concat([combined_df, sheet_dic[key]])
    
    # Create list of dates in csv
    date_df = combined_df.loc[combined_df['Close'].isna()]
    date = list(date_df['Time Interval'])
    
    # Clean list of dates such that its format is m/d/y
    dateobj = []
    for item in date:
        dateobj.append(datetime.datetime.strptime(item[:-16], "%d%b%Y"))
    dateseries = pd.Series(dateobj)
    dateseries.to_csv('datelist.csv')

    # Create list of only rows with actual data
    etf_pricing_df = combined_df[combined_df['Close'].notna()]
    etf_pricing_df = etf_pricing_df[etf_pricing_df['Time Interval'] != "Summary"] #Use this line if getting all time intervals
    etf_pricing_df = etf_pricing_df.reset_index()

    # Set each row to have its own date
    date_iter = pd.Series(dateseries).iteritems()
    cur_date = ""
    count = 0
    for (i, row) in etf_pricing_df.iterrows():
        if(row['Time Interval'] == "09:30 - 10:00"):
            cur_date = dateseries[count]
            count += 1
        etf_pricing_df.at[i,'Date'] = cur_date

    # Add Day of week based on date
    etf_pricing_df['Weekday'] = etf_pricing_df['Date'].apply(lambda x: x.weekday())

    # Thiss is true
    rows_date_to_remove = etf_pricing_df.loc[etf_pricing_df['Net Chg']==0]
    rows_date_to_remove = rows_date_to_remove[rows_date_to_remove['Tick Count']==1]
    dates_to_remove = rows_date_to_remove['Date']
    etf_pricing_df = etf_pricing_df.loc[~etf_pricing_df['Date'].isin(dates_to_remove)]
     # Use this line if getting all time intervals
    
    return etf_pricing_df

open_close_df = clean_date(prices_df) #returns grouped df
### REMOVING STUPID DAYS
open_close_df = open_close_df

open_close_df.to_csv('USO_date_cleaned.csv')

In [53]:
open_close_df

,index,Time Interval,Close,Net Chg,Open,High,Low,Tick Count,Volume,Date,Weekday
0,2,09:30 - 10:00,315.84,10.96,313.92,316.56,313.92,4153.0,214532,2012-01-03,1
1,3,10:00 - 10:30,316.24,0.40,315.84,316.88,315.68,3300.0,142334,2012-01-03,1
2,4,10:30 - 11:00,315.76,-0.48,316.32,317.04,315.52,3308.0,125102,2012-01-03,1
3,5,11:00 - 11:30,314.48,-1.28,315.76,316.24,314.48,2321.0,78503,2012-01-03,1
4,6,11:30 - 12:00,315.44,0.96,314.48,315.68,314.16,2486.0,77054,2012-01-03,1
...,...,...,...,...,...,...,...,...,...,...,...
33088,528,13:30 - 14:00,65.11,-0.06,65.18,65.30,65.01,1994.0,458381.0,2022-02-25,4
33089,529,14:00 - 14:30,65.31,0.20,65.11,65.50,65.08,2457.0,471463.0,2022-02-25,4
33090,530,14:30 - 15:00,65.48,0.17,65.35,65.69,65.29,1336.0,293893.0,2022-02-25,4
33091,531,15:00 - 15:30,65.73,0.25,65.48,65.76,65.47,1240.0,238147.0,2022-02-25,4
